# Options Screener

This notebook is meant to analyze a list of tickers and output which tickers have the highest value call options with the highest probability of increasing in value in the short-term.  

Manual steps procedure:  
- Update options_list2.csv with the latest OI pct changes from Optionistics.com using the options_list_compare.xlsx file  
- Get the latest finviz file from the local directory  

This screener looks at the following attributes:  
- Whether an option is 'cheap' i.e. has low realized volatility compared to its historical realized volatility, the lowest 15% are given a point  
- Implied volatility premium or discount, the top 15% are given a point  
- Open interest % change from optionistics, these are given a point  
- Highest float short percent, the top 15% are given a point

### REMEMBER TO UPDATE THE OPTIONS_LIST2.CSV MANUALLY, BE SURE TO LOG IN FIRST.

In [1]:
# %matplotlib inline

import math
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
import sys

In [2]:
import time

tic = time.perf_counter()

In [3]:
import pandas as pd

tickers_df = pd.read_csv('../0_inputs/asset_screeners/options_list1.csv')
# optionistics_data = pd.read_csv('../0_inputs/asset_screeners/options_list2.csv')
# optionistics_data.head(3)

ticker_dictionary = {
    'VIX':'^VIX'
}

# ticker_list_df = pd.merge(tickers_df[['ticker']],optionistics_data[['ticker']],on='ticker',how='outer')

ticker_list = tickers_df['ticker'].to_list()

In [4]:
def realized_vol(price_data, window=30):
    log_return = (price_data["Close"] / price_data["Close"].shift(1)).apply(np.log)
    return log_return.rolling(window=window, center=False).std() * math.sqrt(252)

In [5]:
# windows = [30, 60, 90, 120] # calendar days
windows = [22, 44, 66, 88] # trading days
quantiles = [0.25, 0.75]

min_ = []
max_ = []
median = []
top_q = []
bottom_q = []
realized = []

In [6]:
completed_tkr_list = []
realized_vol_list = []
avg_pct_list = []
pct30_list = []
pct60_list = []
pct90_list = []
pct120_list = []
counter = 1

cur_price_dict = {}

# ticker_list = ['AAPL','TSLA','MSFT'] # test list

for ticker in ticker_list:

    print(f"Processing {ticker}, {counter} of {len(ticker_list)}...")
    
    try:
        # data = yf.download("SPY AAPL", start="2017-01-01", end="2017-04-30")

        # start="2020-01-01", end="2020-12-31"

        # start_date = '2020-01-01'
        # end_date = '2020-12-31'

        if ticker in ticker_dictionary:
            tkr = ticker_dictionary[ticker]
        else: tkr = ticker

        data = yf.download(  # or pdr.get_data_yahoo(...
                # tickers list or string as well
                tickers = tkr,

                # use "period" instead of start/end
                # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
                # (optional, default is '1mo')
                period = "1y",

        #         start = start_date,
        #         end = end_date,

                # fetch data by interval (including intraday if period < 60 days)
                # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
                # (optional, default is '1d')
                interval = "1d",

                # group by ticker (to access via data['SPY'])
                # (optional, default is 'column')
                group_by = 'ticker',

                # adjust all OHLC automatically
                # (optional, default is False)
                auto_adjust = True,

                # download pre/post regular market hours data
                # (optional, default is False)
                prepost = True,

                # use threads for mass downloading? (True/False/Integer)
                # (optional, default is True)
                threads = True,

                # proxy URL scheme use use when downloading?
                # (optional, default is None)
                proxy = None
            )

        # data.to_csv('BTC_1hr_prices.csv')
        # data.head(3)
    #     print(data.tail(5))

        cur_price_dict[ticker] = data['Close'][-1]

        min_ = []
        max_ = []
        median = []
        top_q = []
        bottom_q = []
        realized = []

        for window in windows:

            # get a dataframe with realized volatility
            estimator = realized_vol(window=window, price_data=data)
    #         print(estimator)

            # append the summary stats to a list
            min_.append(estimator.min())
            max_.append(estimator.max())
            median.append(estimator.median())
            top_q.append(estimator.quantile(quantiles[1]))
            bottom_q.append(estimator.quantile(quantiles[0]))
            realized.append(estimator[-1])
    #         print(min_)

        pct_list = []

        pct30 = (realized[0] - bottom_q[0]) / (top_q[0] - bottom_q[0]) * 100
        pct_list.append(pct30)
        pct60 = (realized[1] - bottom_q[1]) / (top_q[1] - bottom_q[1]) * 100
        pct_list.append(pct60)
        pct90 = (realized[2] - bottom_q[2]) / (top_q[2] - bottom_q[2]) * 100
        pct_list.append(pct90)
        pct120 = (realized[3] - bottom_q[3]) / (top_q[3] - bottom_q[3]) * 100
        pct_list.append(pct120)

    #     print(pct30)
    #     print(pct60)
    #     print(pct90)
    #     print(pct120)
    #     print(pct_list)

        avg_pct = sum(pct_list)/len(pct_list)
    #     print(avg_pct)

        completed_tkr_list.append(ticker)
        realized_vol_list.append(round(realized[1] * 100,2)) # this is realized vol for 60 day window
        avg_pct_list.append(round(avg_pct,2))
        pct30_list.append(round(pct30,2))
        pct60_list.append(round(pct60,2))
        pct90_list.append(round(pct90,2))
        pct120_list.append(round(pct120,2))

        counter += 1
        
    except Exception as e:
        print(f'{tkr} was not found.')
        print(f'Exception: {e}')
        print(f'Error:\n{sys.exc_info()}')
        counter += 1

Processing SPY, 1 of 254...
[*********************100%***********************]  1 of 1 completed
Processing QQQ, 2 of 254...
[*********************100%***********************]  1 of 1 completed
Processing SPXU, 3 of 254...
[*********************100%***********************]  1 of 1 completed
Processing AAPL, 4 of 254...
[*********************100%***********************]  1 of 1 completed
Processing AMZN, 5 of 254...
[*********************100%***********************]  1 of 1 completed
Processing TSLA, 6 of 254...
[*********************100%***********************]  1 of 1 completed
Processing AMD, 7 of 254...
[*********************100%***********************]  1 of 1 completed
Processing VIX, 8 of 254...
[*********************100%***********************]  1 of 1 completed
Processing IWM, 9 of 254...
[*********************100%***********************]  1 of 1 completed
Processing HYG, 10 of 254...
[*********************100%***********************]  1 of 1 completed
Processing META, 11 of 25

[*********************100%***********************]  1 of 1 completed
Processing DS, 85 of 254...
[*********************100%***********************]  1 of 1 completed
Processing CTLP, 86 of 254...
[*********************100%***********************]  1 of 1 completed
Processing VRM, 87 of 254...
[*********************100%***********************]  1 of 1 completed
Processing HGEN, 88 of 254...
[*********************100%***********************]  1 of 1 completed
Processing AGEN, 89 of 254...
[*********************100%***********************]  1 of 1 completed
Processing ACRX, 90 of 254...
[*********************100%***********************]  1 of 1 completed
Processing NMTR, 91 of 254...
[*********************100%***********************]  1 of 1 completed
Processing CRNT, 92 of 254...
[*********************100%***********************]  1 of 1 completed
Processing OEG, 93 of 254...
[*********************100%***********************]  1 of 1 completed
Processing THCA, 94 of 254...
[*************

[*********************100%***********************]  1 of 1 completed
Processing NLOK, 167 of 254...
[*********************100%***********************]  1 of 1 completed
Processing DAL, 168 of 254...
[*********************100%***********************]  1 of 1 completed
Processing GLBS, 169 of 254...
[*********************100%***********************]  1 of 1 completed
Processing CLF, 170 of 254...
[*********************100%***********************]  1 of 1 completed
Processing ADVM, 171 of 254...
[*********************100%***********************]  1 of 1 completed
Processing AEM, 172 of 254...
[*********************100%***********************]  1 of 1 completed
Processing TEF, 173 of 254...
[*********************100%***********************]  1 of 1 completed
Processing IPOD, 174 of 254...
[*********************100%***********************]  1 of 1 completed
Processing GFI, 175 of 254...
[*********************100%***********************]  1 of 1 completed
Processing AQMS, 176 of 254...
[****

[*********************100%***********************]  1 of 1 completed
Processing MRO, 249 of 254...
[*********************100%***********************]  1 of 1 completed
Processing TTOO, 250 of 254...
[*********************100%***********************]  1 of 1 completed
Processing STWD, 251 of 254...
[*********************100%***********************]  1 of 1 completed
Processing CVE, 252 of 254...
[*********************100%***********************]  1 of 1 completed
Processing RKT, 253 of 254...
[*********************100%***********************]  1 of 1 completed
Processing HMLP, 254 of 254...
[*********************100%***********************]  1 of 1 completed


In [7]:
import pandas as pd

pct_dictionary = {
    'ticker':completed_tkr_list,
    'realized_vol':realized_vol_list,
    'avg_pct':avg_pct_list,
    'pct30':pct30_list,
    'pct60':pct60_list,
    'pct90':pct90_list,
    'pct120':pct120_list
}

cheap_df = pd.DataFrame(pct_dictionary)
cheap_df = cheap_df.sort_values(['avg_pct'])
cheap_df.to_csv('../0_data_dump/optionistics/realized_vol.csv')
print(f'Size of cheap_df is {len(cheap_df)}')
cheap_df

Size of cheap_df is 254


,ticker,realized_vol,avg_pct,pct30,pct60,pct90,pct120
5,TSLA,43.26,-187.76,-59.29,-198.17,-257.70,-235.89
235,RNWK,13.57,-161.04,-194.85,-245.12,-89.40,-114.78
196,M,54.16,-139.69,-29.25,-46.03,-406.80,-76.68
46,IPOF,1.74,-129.37,-5.28,-50.64,-81.33,-380.24
29,UVXY,79.00,-127.47,-11.10,-35.20,-320.06,-143.51
...,...,...,...,...,...,...,...
78,NLY,36.65,190.01,295.34,193.71,115.19,155.79
104,KGC,54.56,196.21,183.51,158.49,172.29,270.54
242,VTNR,162.78,252.58,13.03,313.51,345.93,337.84
65,MNMD,202.39,361.63,524.35,287.72,295.30,339.15


In [8]:
import glob
import os

# Get the latest file from the directory
list_of_files = glob.glob('../0_inputs/asset_screeners/Finviz_Data/*') # * means all if need specific format then *.csv
latest_file = max(list_of_files, key=os.path.getctime)
print(latest_file)

../0_inputs/asset_screeners/Finviz_Data\finviz (75).csv


In [9]:
import pandas as pd
import numpy as np

col_names = {
    'Ticker':'ticker',
    'Float Short':'float_short'
}

# Bring finviz.csv file into dataframe
finviz_df = pd.read_csv(latest_file)
finviz_df = finviz_df.rename(columns=col_names)
finviz_df = finviz_df[['ticker','float_short']]
finviz_df['float_short'] = finviz_df['float_short'].str.slice(0,-1) # remove the % character from the end of the string
finviz_df['float_short'] = finviz_df['float_short'].astype(str).astype(float) # convert to float for proper sorting
finviz_df = finviz_df.sort_values(['float_short'],ascending=False)

# Add Relative Volume Logic here
finviz_df['float_short_pt'] = 0
finviz_df['float_short_pt'] = np.select(condlist=[(finviz_df['float_short']>finviz_df['float_short'].quantile(0.85))],
                           choicelist=[1],
                           default=0)

# finviz_df = finviz_df[finviz_df['float_short'] > finviz_df['float_short'].quantile(.85)]
# finviz_df2 = finviz_df[finviz_df['float_short_pt']==1]
print(f'Size of finviz_df is {len(finviz_df)}.')
finviz_df.head(3)

Size of finviz_df is 8508.


,ticker,float_short,float_short_pt
126,ADTX,130.15,1
6215,PXMD,59.89,1
437,APRN,52.78,1


In [10]:
# Optional for if internet is running slow
# cheap_df = pd.read_csv('../0_data_dump/optionistics/realized_vol_output.csv')
# cheap_df = cheap_df.drop('Unnamed: 0', axis=1)

# Add Relative Volume Logic here
cheap_df['cheap_pt'] = 0
cheap_df['cheap_pt'] = np.select(condlist=[(cheap_df['avg_pct'] < 0)],
                           choicelist=[1],
                           default=0)

print(f'Size of cheap_df is {len(cheap_df)}')
cheap_df.head(3)

Size of cheap_df is 254


,ticker,realized_vol,avg_pct,pct30,pct60,pct90,pct120,cheap_pt
5,TSLA,43.26,-187.76,-59.29,-198.17,-257.7,-235.89,1
235,RNWK,13.57,-161.04,-194.85,-245.12,-89.4,-114.78,1
196,M,54.16,-139.69,-29.25,-46.03,-406.8,-76.68,1


Implied volatility premium or discount calculations here.

In [11]:
import yfinance as yf
import datetime
import sys

today_date = datetime.datetime.now()
target_date = today_date + datetime.timedelta(days=75) # calendar days

# ticker_list = ['AAPL','TSLA','MSFT'] # test list

ticker_list2 = []
expiry_list = []
strike_list = []
iv_list = []
counter = 1

for ticker in ticker_list:
    
    try:
        print(f'\nProcessing {ticker}, {counter} of {len(ticker_list)}...')
        
        # options_data = yf.Ticker("MSFT") # test input
        options_data = yf.Ticker(ticker)
        option_dates_list = options_data.options

        closest_date = today_date

        for x in option_dates_list:
            x = datetime.datetime.strptime(x, '%Y-%m-%d')
        #     print(x,',',(x-target_date).days)
            if (abs((x - target_date).days)) < abs(((closest_date - target_date).days)):
                closest_date = x

        # print(closest_date)
        # days_to_expiration = (closest_date - today_date).days
        # print(days_to_expiration)
        closest_date = closest_date.strftime("%Y-%m-%d")
#         print(f'Closest expiry is: {closest_date}')

        opt = options_data.option_chain(closest_date)
        strike_price_np_array = opt[0]['strike'].to_numpy()

        if ticker in cur_price_dict:
            cur_price = cur_price_dict[ticker]
        else:
            data = yf.download(
                tickers = tkr,
                period = "5d",
                interval = "1d",
                group_by = 'ticker',
                auto_adjust = True,
                prepost = True,
                threads = True,
                proxy = None
            )
            cur_price = data['Close'][-1]
        
#         cur_price = 280 # test input
        closest_strike = 0

        strike_ctr = 0
        
        for x in strike_price_np_array:
            if abs(cur_price - x) < abs(cur_price - closest_strike):
                closest_strike = x
                strike_index = strike_ctr
            strike_ctr += 1

#         print(f'Closest strike price is: {closest_strike}')

        strike_df = opt[0][opt[0]['strike']==closest_strike]
        implied_vol = round(strike_df['impliedVolatility'][strike_index] * 100,2)
#         print(f'Implied volatility is: {implied_vol}')

        ticker_list2.append(ticker)
        expiry_list.append(closest_date)
        strike_list.append(closest_strike)
        iv_list.append(implied_vol)
        counter += 1
    
    except Exception as e:
        print(f'{ticker}, {counter} not found.')
        print(f'Exception is {e}')
        print(f'Error is: {sys.exc_info()}')
        counter += 1


Processing SPY, 1 of 254...
Closest expiry is: 2022-12-16
Closest strike price is: 357.0
Implied volatility is: 30.54

Processing QQQ, 2 of 254...
Closest expiry is: 2022-12-16
Closest strike price is: 268.0
Implied volatility is: 36.78

Processing SPXU, 3 of 254...
Closest expiry is: 2022-12-16
Closest strike price is: 22.0
Implied volatility is: 84.96

Processing AAPL, 4 of 254...
Closest expiry is: 2022-12-16
Closest strike price is: 140.0
Implied volatility is: 42.34

Processing AMZN, 5 of 254...
Closest expiry is: 2022-12-16
Closest strike price is: 113.0
Implied volatility is: 50.82

Processing TSLA, 6 of 254...
Closest expiry is: 2022-12-16
Closest strike price is: 266.67
Implied volatility is: 67.17

Processing AMD, 7 of 254...
Closest expiry is: 2022-12-16
Closest strike price is: 65.0
Implied volatility is: 60.47

Processing VIX, 8 of 254...
Closest expiry is: 2022-09-30
VIX, 8 not found.
Exception is Expiration `2022-09-30` cannot be found. Available expiration are: []
Erro

Closest strike price is: 2.5
Implied volatility is: 0.0

Processing ANY, 67 of 254...
Closest expiry is: 2022-12-16
Closest strike price is: 0.5
Implied volatility is: 215.63

Processing BIOR, 68 of 254...
Closest expiry is: 2022-11-18
Closest strike price is: 0.5
Implied volatility is: 121.88

Processing RIG, 69 of 254...
Closest expiry is: 2022-12-16
Closest strike price is: 2.5
Implied volatility is: 89.45

Processing PFE, 70 of 254...
Closest expiry is: 2022-12-16
Closest strike price is: 45.0
Implied volatility is: 31.15

Processing MMAT, 71 of 254...
Closest expiry is: 2022-11-18
Closest strike price is: 0.5
Implied volatility is: 215.63

Processing C, 72 of 254...
Closest expiry is: 2022-12-16
Closest strike price is: 42.5
Implied volatility is: 39.55

Processing MULN, 73 of 254...
Closest expiry is: 2022-11-18
Closest strike price is: 0.5
Implied volatility is: 253.13

Processing CGC, 74 of 254...
Closest expiry is: 2022-12-16
Closest strike price is: 2.5
Implied volatility is:

Closest expiry is: 2022-11-18
Closest strike price is: 38.0
Implied volatility is: 24.76

Processing STLA, 124 of 254...
Closest expiry is: 2022-12-16
STLA, 124 not found.
Exception is ('Connection aborted.', TimeoutError(10060, 'A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond', None, 10060, None))
Error is: (<class 'requests.exceptions.ConnectionError'>, ConnectionError(ProtocolError('Connection aborted.', TimeoutError(10060, 'A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond', None, 10060, None))), <traceback object at 0x000001636A5F5708>)

Processing DNA, 125 of 254...
DNA, 125 not found.
Exception is ('Connection aborted.', TimeoutError(10060, 'A connection attempt failed because the connected party did not properly re

Closest expiry is: 2022-12-16
Closest strike price is: 25.0
Implied volatility is: 56.69

Processing PTON, 152 of 254...
Closest expiry is: 2022-12-16
Closest strike price is: 7.0
Implied volatility is: 122.27

Processing PIPP, 153 of 254...
Closest expiry is: 2023-01-20
Closest strike price is: 10.0
Implied volatility is: 7.23

Processing TRVG, 154 of 254...
Closest expiry is: 2022-11-18
Closest strike price is: 0
TRVG, 154 not found.
Exception is 1
Error is: (<class 'KeyError'>, KeyError(1), <traceback object at 0x0000016369273B88>)

Processing CLOV, 155 of 254...
Closest expiry is: 2022-11-18
Closest strike price is: 1.5
Implied volatility is: 103.91

Processing OXY, 156 of 254...
Closest expiry is: 2022-12-16
Closest strike price is: 62.5
Implied volatility is: 57.73

Processing MPW, 157 of 254...
Closest expiry is: 2022-12-16
Closest strike price is: 12.0
Implied volatility is: 70.9

Processing ACB, 158 of 254...
Closest expiry is: 2022-12-16
Closest strike price is: 1.0
Implied v

Closest expiry is: 2022-12-16
Closest strike price is: 5.0
Implied volatility is: 594.14

Processing XSPA, 213 of 254...
Closest expiry is: 2022-11-18
Closest strike price is: 1.0
Implied volatility is: 106.25

Processing DRRX, 214 of 254...
Closest expiry is: 2023-01-20
Closest strike price is: 0
DRRX, 214 not found.
Exception is 0
Error is: (<class 'KeyError'>, KeyError(0), <traceback object at 0x000001636A372248>)

Processing SFT, 215 of 254...
Closest expiry is: 2022-12-16
Closest strike price is: 0
SFT, 215 not found.
Exception is 0
Error is: (<class 'KeyError'>, KeyError(0), <traceback object at 0x000001636A60AB48>)

Processing GDRX, 216 of 254...
Closest expiry is: 2022-11-18
Closest strike price is: 5.0
Implied volatility is: 92.58

Processing BEST, 217 of 254...
Closest expiry is: 2023-01-20
Closest strike price is: 0
BEST, 217 not found.
Exception is 1
Error is: (<class 'KeyError'>, KeyError(1), <traceback object at 0x000001636938C988>)

Processing UXIN, 218 of 254...
Closest

In [12]:
print('Completed.')

Completed.


Create a dataframe for the implied volatility data.

In [13]:
iv_dictionary = {
    'ticker':ticker_list2,
    'expiry':expiry_list,
    'strike':strike_list,
    'implied_vol':iv_list
}

iv_df = pd.DataFrame(iv_dictionary)
iv_df["implied_vol"] = iv_df["implied_vol"].astype(str).astype(float)
iv_df = iv_df.sort_values(['implied_vol'],ascending=False)
iv_df.to_csv('../0_data_dump/optionistics/implied_vols.csv')
print(f'Size of iv_df is {len(iv_df)}.')
iv_df.head(3)

Size of iv_df is 208.


,ticker,expiry,strike,implied_vol
175,ATUS,2022-12-16,5.0,594.14
186,ATAI,2022-11-18,2.5,357.81
68,MULN,2022-11-18,0.5,253.13


In [14]:
print('Completed.')

Completed.


In [15]:
# for col in final_df.columns:
#     print(col)

In [16]:
final_df = pd.merge(cheap_df,finviz_df,on='ticker',how='left')
print(len(final_df))
# final_df = pd.merge(final_df,optionistics_data,on='ticker',how='outer')
# print(len(final_df))
final_df = pd.merge(final_df,iv_df,on='ticker',how='outer')
print(len(final_df))

final_df['iv_premium'] = final_df['implied_vol'] - final_df['realized_vol']

final_df['iv_prem_pt'] = 0
final_df['iv_prem_pt'] = np.select(condlist=[(final_df['iv_premium']>final_df['iv_premium'].quantile(0.85))],
                           choicelist=[1],
                           default=0)

# final_df['oi_pct_chg_pts'] = final_df['oi_pct_chg_pts'].fillna(0)
final_df['cheap_pt'] = final_df['cheap_pt'].fillna(0)
final_df['total_pts'] = final_df['cheap_pt'] + final_df['float_short_pt'] + final_df['iv_prem_pt']
final_df = final_df[['ticker','expiry','strike','total_pts','iv_prem_pt','cheap_pt','float_short_pt','iv_premium','avg_pct','float_short']]
final_df = final_df.sort_values(by = ['total_pts','float_short'],ascending=[False,False])
final_df = final_df[final_df['total_pts'] > 0]
final_df.to_csv('../0_data_dump/asset_screener/options_output.csv')
final_df.head(20)

254
254


,ticker,expiry,strike,total_pts,iv_prem_pt,cheap_pt,float_short_pt,iv_premium,avg_pct,float_short
64,SKLZ,2022-11-18,1.0,3.0,1,1,1.0,35.92,-20.67,17.94
76,CLSK,2022-12-16,2.5,3.0,1,1,1.0,42.86,-12.32,14.97
79,MULN,2022-11-18,0.5,3.0,1,1,1.0,160.88,-9.50,13.78
22,GNUS,2022-11-18,0.5,3.0,1,1,1.0,106.30,-57.54,11.85
87,MMAT,2022-11-18,0.5,3.0,1,1,1.0,139.13,-3.84,11.04
52,BITF,2022-12-16,1.0,3.0,1,1,1.0,36.70,-27.30,8.94
50,IDEX,2022-11-18,0.5,3.0,1,1,1.0,151.69,-29.44,8.66
28,SIRI,2022-12-16,5.0,2.0,0,1,1.0,-20.45,-49.26,33.38
17,NKLA,2022-11-18,4.0,2.0,0,1,1.0,21.07,-62.11,30.50
175,CLVS,2022-11-18,1.0,2.0,1,0,1.0,90.67,59.52,23.29


In [17]:
toc = time.perf_counter()
total_time = toc - tic

print(f'Program completed in {total_time:.2f} seconds or {total_time/60:.2f} minutes on {datetime.datetime.now()}.')

Program completed in 404.39 seconds or 6.74 minutes on 2022-09-30 20:48:35.738433.
